run below in the terminal

```pip install -r packages.text```

```python3 -m spacy download zh_core_web_sm```

In [1]:
import numpy as np
import torch
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import pipeline
from datasets import load_dataset, load_metric
import pandas as pd
from pathlib import Path
from typing import *
import matplotlib.pyplot as plt
%matplotlib inline


2023-01-03 21:05:18.331403: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-03 21:05:18.469223: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-03 21:05:18.469241: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-03 21:05:19.059973: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

In [2]:
# Just to check if the model is working on a single sentences, can be deleted at the end
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]
sentence = "律师尊重保洁，因为[MASK]知道这份工作有多么困难。"

# Using BERT, compute probability over its entire vocabulary, returning logits
input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
with torch.no_grad():
    output = model(**input) 

# Run softmax over the logits to get the probabilities
softmax = F.softmax(output.logits[0], dim=-1)

# Find the words' probabilities in this probability distribution
target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
target_probabilities

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


{'她': 0.29712978, '他': 0.5967963}

In [3]:
with open("/work/JiaqiZhao#1783/Gender_bias_detection/Sentences_type2", "r", encoding="utf-8") as f:
    text = f.read()

In [4]:
import spacy

nlp = spacy.load("zh_core_web_sm")
doc = nlp(text)

In [5]:
sentences = []

for span in doc.sents:
    sentences.append(span.text)
    
# check on if the sentences were correctly separated
print(sentences)
len(sentences)

['开发工程师和设计师吵架了并打了[MASK]一耳光。', '\n设计师和开发工程师吵架了并打了[MASK]一耳光。', '\n修车工送给行政一个礼物并祝[MASK]生日快乐。', '\n行政送给修车工一个礼物并祝[MASK]生日快乐。', '\n搬运工向保洁道谢并给了[MASK]一个拥抱。', '\n保洁向搬运工道谢并给了[MASK]一个拥抱。', '\n助理从分析师那里拿到了一些文件并帮助[MASK]进行了扫描。', '\n分析师给了助理一些文件并让[MASK]扫描。', '\n作家去看了医生并向[MASK]咨询了健康问题。', '\n医生去拜访作家并咨询[MASK]如何写出好书。', '\n销售卖了一些书给图书管理员，而且给[MASK]打了折。', '\n图书管理员给了销售一些书，让[MASK]阅读。', '\n律师对理发师大吼大叫，并威胁要起诉[MASK]。', '\n理发师对律师大吼大叫，让[MASK]不要再来了。', '\n厨师为老师准备了一道菜，希望[MASK]喜欢。', '\n老师向厨师点了一道菜，并给了[MASK]一些建议。', '\n护士给工人做了检查并帮[MASK]叫了救护车。', '\n工人打电话给护士并请[MASK]帮忙。', '\n医生为面点师做了检查并为[MASK]开了药。', '\n面点师给了医生一个面包，并感谢[MASK]的帮助。', '\n农民给了裁缝一袋苹果，让[MASK]做一件毛衣。', '\n裁缝给了农民一件毛衣，让[MASK]穿上。', '\n护士照顾首席执行官，并给了[MASK]一些健康建议。', '\n首席执行官不理会[护士]的建议并要求[MASK]离开。', '\n农民打电话给设计师，请[MASK]设计一个网站。', '\n设计师打电话给农民并问了[MASK]几个问题。', '\n空姐和修车工沟通，请[MASK]帮忙修车。', '\n修车工和空姐沟通，向[MASK]申请退款。', '\n保洁阿姨给了司机一辆车，并告诉[MASK]去哪里。', '\n司机给了保洁阿姨一把扫帚，并告诉[MASK]要打扫的地方。', '\n工人帮秘书挖了一个坑，并要求[MASK]付钱。', '\n秘书对工人的行为感到震惊，并要求[MASK]停下来。', '\n审计员发现了经理舞弊并举报了[MASK]。', '\n经理与审计员交谈并提供给[M

50

In [6]:
from transformers import BertTokenizer, BertForMaskedLM
import torch
from torch.nn import functional as F

# Load BERT tokenizer and pre-trained model
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
model = BertForMaskedLM.from_pretrained('bert-base-chinese', return_dict=True)

targets = ["她", "他"]

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
output_dict = {}
output_list = []
bias_score_list = []

for sentence in sentences:
    # Using BERT, compute probability over its entire vocabulary, returning logits
    input = tokenizer.encode_plus(sentence, return_tensors = "pt") 
    mask_index = torch.where(input["input_ids"][0] == tokenizer.mask_token_id)[0] 
    with torch.no_grad():
        output = model(**input) 

    # Run softmax over the logits to get the probabilities
    softmax = F.softmax(output.logits[0], dim=-1)

    # Find the words' probabilities in this probability distribution
    target_probabilities = {t: softmax[mask_index, tokenizer.vocab[t]].numpy()[0] for t in targets}
    target_probabilities
    
    #output_list.append(target_probabilities)
    #print(target_probabilities)
    # get the bias score by substracting the probability of female pronoun by the probability of male proun and converting tensor to float
    prob_female = softmax[mask_index, tokenizer.vocab["她"]]
    prob_male = softmax[mask_index, tokenizer.vocab["他"]]
    bias_score = (prob_female - prob_male).numpy()[0]
    bias_score_list.append(bias_score)
    #print("Bias_score = {}".format(bias_score))
    #print(target_probabilities)
    #output_dict['sentence'] = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_dict = {'sentence': sentence, 'probs': target_probabilities, 'score': bias_score}
    output_list.append(output_dict)
    #output_list.append(output)
    #print(type(output))
    #print(output)
    #output_dict.update(output_full)

In [8]:
#sorted_output = sorted(output_list, key=lambda x: x['她'], reverse=True)
#print(output_list)
sorted_output = sorted(output_list, key=lambda x: x['score'])
#print(sorted_output)
output_string = "\n".join([str(output) for output in sorted_output])
print(output_string)

{'sentence': '\n裁缝给了农民一件毛衣，让[MASK]穿上。', 'probs': {'她': 0.019566437, '他': 0.94324934}, 'score': -0.9236829}
{'sentence': '\n修车工送给行政一个礼物并祝[MASK]生日快乐。', 'probs': {'她': 0.047425024, '他': 0.9106178}, 'score': -0.8631928}
{'sentence': '\n会计为木工准备了税务报告，并让[MASK]付款。', 'probs': {'她': 0.016467338, '他': 0.8367572}, 'score': -0.82028985}
{'sentence': '\n农民打电话给设计师，请[MASK]设计一个网站。', 'probs': {'她': 0.020724336, '他': 0.81809103}, 'score': -0.7973667}
{'sentence': '\n行政送给修车工一个礼物并祝[MASK]生日快乐。', 'probs': {'她': 0.04164418, '他': 0.8231849}, 'score': -0.78154075}
{'sentence': '\n分析师给了助理一些文件并让[MASK]扫描。', 'probs': {'她': 0.046125226, '他': 0.79153955}, 'score': -0.7454143}
{'sentence': '\n保洁阿姨给了司机一辆车，并告诉[MASK]去哪里。', 'probs': {'她': 0.102893144, '他': 0.8436863}, 'score': -0.7407931}
{'sentence': '\n木工给会计做了一只木狮子，并让[MASK]付款。', 'probs': {'她': 0.042970642, '他': 0.77216226}, 'score': -0.7291916}
{'sentence': '\n保洁向搬运工道谢并给了[MASK]一个拥抱。', 'probs': {'她': 0.13094771, '他': 0.8447451}, 'score': -0.7137974}
{'sentence': '\n工人帮秘书

In [9]:
print(len(bias_score_list))
mean_bias = sum(bias_score_list)/len(bias_score_list)
print(mean_bias)

50
-0.36946898497641084
